# Exploratory Data Analysis (EDA)

## Objective
The goal of this notebook is to understand the structure of the Home Credit Default Risk dataset, explore key drivers of loan default, and identify data quality issues that will inform feature engineering and modeling decisions.

This analysis focuses on:
- Target variable distribution and class imbalance
- Key applicant characteristics related to default risk
- Missing data patterns and potential signal in missingness
- Early insights that guide modeling choices

---

## Dataset Overview
Primary dataset:
- `application_train.csv`: Loan application data with default labels (`TARGET`)

Key target definition:
- `TARGET = 1`: Client defaulted on loan
- `TARGET = 0`: Client repaid loan

---

## Key Questions
1. What proportion of applicants default?
2. Which applicant features show clear separation between defaulters and non-defaulters?
3. How severe is class imbalance?
4. Which variables have high missingness, and is missingness informative?
5. Are there obvious data quality issues (outliers, impossible values)?

---

## EDA Plan
1. Load and inspect data
2. Examine target distribution
3. Analyze numeric features vs target
4. Analyze categorical features vs target
5. Investigate missing values
6. Summarize findings and modeling implications


## 1. Load and Inspect Data